<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Introducción a Redes Neuronales Recurrentes</center></span>

<center> Ejemplo 1: Serie de tiempo simulada</center>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
3. [Dive into Deep Learnig](https://d2l.ai/)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Series de tiempo y aprendizaje profundo](#Series-de-tiempo-y-aprendizaje-profundo)
* [Modelo recurrente](Modelo-recurrente)
* [Importar módulos requeridos](#Importar-módulos-requeridos)
* [Datos sintéticos](#Datos-sintéticos)
* [Modelo LSTM](#Modelo-LSTM)
* [Modelo GRU](#Modelo-GRU)

## <span style="color:blue">Referencias</span>

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)

## <span style="color:blue">Introducción</span> 

Los datos de la serie temporal introducen una "dependencia dura" de los pasos de tiempo anteriores, por lo que no se cumple la independencia de las observaciones. ¿Cuáles son algunas de las propiedades que puede tener una serie temporal?

La *estacionalidad* y la *autocorrelación* son algunas de las propiedades de la serie temporal en las que puede estar interesado.

Se dice que una serie de tiempos es **estacionaria** cuando la media y la varianza permanecen constantes en el tiempo. 

Una serie temporal tiene una **tendencia** si la media varía con el tiempo. A menudo puede eliminarlo y hacer que la serie sea estacionaria aplicando transformación(es) logarítmicas de los datos.


**La estacionalidad** se refiere al fenómeno de las variaciones en plazos específicos. por ejemplo, personas que compran más árboles de Navidad durante Navidad (quién lo hubiera pensado). Un enfoque común para eliminar la estacionalidad es usar la diferenciación.

**La autocorrelación** se refiere a la correlación entre el valor actual con una copia de un tiempo anterior (retraso).  

Usaremos redes neuronales para el modelamiento de esta series.

## <span style="color:blue">Series de tiempo y aprendizaje profundo</span> 

Una serie de tiempo univariada es una sucesión de valores en el tiempo, digamos

$$
x_1, x_2, x_3, \ldots x_T.
$$

Por ejemplo consideremos la serie dada pos los siguientes valores: 

$$\{3.4, 5.2, 4.6, 6.2, 5.5, 4.0, 7.2, 8.1, 6.9, 9.2, 9.5, 9.8, 8.9, 9.6, 9.7, 9.9 \}
$$


El problema de hacer pronósticos a partir de una serie de tiempo puede verse como un problema de aprendizaje profundo , si la información se organiza inicialmente como un problema de regresión o de clasificación. La siguiente imagen ilustra el procedimiento más común.

<figure>
<img src="../Imagenes/series.jpg"  width="800" height="800" align="center"/> 
</figure>

En la imagen se observan varias cosas interesantes.

1. Los datos de la serie se han organizado como si se tuviera un regresión. La matriz de diseño $X$ está constituida por filas con 8 columnas. El número de columnas corresponde al rezago (núnro de pasos hacia atrás en la serie para poder predecir el sigueinte valor.
2. La variable de interés $Y$ es la variable que se quiere predecir.

Observe que para la primera fila se tomaron los primeros 8 datos de la serie y el valor de la variable a predecir es la novena observación. En la segunda fila nos hemos desplazado un casilla a la derecha en la serie.

Puesto de esta forma, cada fila de la matriz $X$ es independiente de las demás y constituye y tensor de dimensión 1 en la entrada de una red neuronal. La variable target es es el tensor $Y$.

Visto de esta forma tenemos un problema de aprendizaje profundo, por lo que un por ejemplo un perceptron podría usarse para construir un modelo predictivo.

### Prediccioón a más de un paso

Si lo que se desea es hacer predicciones, varios pasos adelante y no del siguiente paso, el único cambio es en la variable objetivo (target). La imagen muestra el caso de predcciones tres pasos adelante. Observe que en esta caso la variable objetivo se toma tre pasos adelante del último elemenot de cada fila. 

Observe además que las dos últimas filas de la matriz no se pueden utilizar, debido a que no tenemos datos más allá de el útlimo dato de la serie.

<figure>
<img src="../Imagenes/prediccion_tres_dias.jpg"  width="800" height="800" align="center"/> 
</figure>

### Series Multivariadas

En algunas situaciones tenemos  más de una serie. La siguiente imagen muestra la nueva situación. 

<figure>
<img src="../Imagenes/Serie_Multivariada.jpg"  width="800" height="800" align="center"/> 
</figure>

Como se observa, ahora los tensores de datos de entrada tiene una dimensión más, la cual en el lenguaje de los modelos convolucionales, que estudiarmeos más adelante, llamaremos canales. En el ejemplo, se muestra como se preparan los datos de tres series de tiempos para tratarlos como un problema de aprendizaje profundo.

La imagen ilustra claramente, la semejanza de los datos de imágenes en donde hay varia canales de color. Esta forma de organización de los datos para aprendizaje profundo llevó a los diseñadores de capas neuronales para este tipo de problemas a considerar tanto en problemas de visión por computador como para redes recurrentes, que los tensores de entrada siempre sean de dimensión 3: largo, ancho y canales. 

En el caso de las series siempre se tendrá 

1. tamaño de batch, por ejemplo 32,
2. longitud del rezago, por ejemplo 8,
3. número de canales, por ejemplo 3.

Adicionalmente puede verse que la variable objetivo tiene por lo menos el tamaño del batch. Este tensor puede tener una segunda dimensión que podría corresponder  al número de canales, pero no es necesario. Por ejemplo, podría tenerse que la variable objetivo podría ser el promedio de los valores predichos de cada acanal.

### Modelos multivariadas multi-cabeza.

Una vez las series de tiempo se presentan como un problema de aprendizaje profundo, es posible pensar en diferentes arquitecturas de redes. La siguiente imagen presenta un alternativa que consiste en que cada serie entran por una camino diferente a la red, sigue un camino separado por un tiempo y luego las predicciones se concadenan para llegar a una predicción final.

<figure>
<img src="../Imagenes/multicabeza.jpg"  width="800" height="800" align="center"/> 
</figure>

## <span style="color:blue">Modelo recurrente</span> 

Este código puede ser usado como base para la implementación de modelos mas complejos. Para un introdución teórica a las redes LSTM consulte el cuaderno [Redes LSTM (Long Short Term Memory Networks)](Intro_LSTM.ipynb). La siguiente imagen muestra la estructura general de una capa recurrente. LSTM es un caso particular y el gráfico ilustra bien la forma de procesamiento.

<figure>
<img src="../Imagenes/modelo_recurrente.jpg"  width="800" height="800" align="center"/> 
</figure>

Comenzaremos con un ejemplo simple de pronosticar los valores de la función Seno utilizando una red LSTM simple.

## <span style="color:blue">Importar módulos requeridos</span> 

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import pandas as pd
#import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
#from tensorflow import keras
#
from tensorflow.keras.layers import Input, LSTM, Dense, GRU
from tensorflow.keras.models import Model
#
from tensorflow.keras.optimizers import Adam
#
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import MinMaxScaler
#
print("Versión de Tensorflow: ", tf.__version__)

### Configuraciones básicas generales

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
#
#sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 16, 10
#

#tf.random.set_seed(RANDOM_SEED)
#


## <span style="color:blue">Datos sintéticos</span>

Vamos a generar unos datos que siguen un comportamiento sinosoidal, con una tendencia ascendiente. Se introduce ruido Gaussiano. 

Modifique las siguientes dos líneas y haga su priopia simulación.

In [ ]:
#
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
trend = .1
#
time = np.arange(0, 100, 0.1)
sin =  trend*time + np.sin(time) + np.random.normal(scale=0.5, size=len(time))

### Primer gráfico. Datos simulados.

In [ ]:
plt.plot(time, sin, label='Función seno con tendencia ascendente y ruido Gaussiano');
plt.title("Serie de tiempo simulada, con la función seno y una tendencia", size = 20)
plt.legend();
plt.show()

In [ ]:
print( 'Número de datos: ', sin.shape[0])

## Preprocesamiento de los datos

### Escala los datos

Introducimos un instancia de clase del escalador *MinMaxScaler* para llevar los datos a la escala $[0,1]$

In [ ]:
#crea un dataset 
#
df1 = pd.DataFrame(sin, index=time, columns=['serie'])
# crea el objeto  scaler y escala los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df1.values)
#
dataset = pd.DataFrame(scaled_data,index=df1.index, columns=['serie'])
#

In [ ]:
dataset.shape

### Gráfico de datos escalados

In [ ]:
#serie_0_1 = dataset.serie_0_1.values

plt.plot(time, dataset, label='Datos estandarizados a escala [0,1]');
plt.title("Serie de tiempo simulada, con la función seno y una tendencia", size = 20)
plt.legend();
plt.show()

###  Separación de datos entrenamiento y de validación

En el caso de las series de tiempo, los datos de entrenamiento se toman desde el comienzo de la serie y los de validación desde el final haci atras. 

Veamos.

In [ ]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset.iloc[0:train_size], dataset.iloc[train_size:len(df1)]
len_train = len(train)
len_test = len(test)
print(len_train, len_test)

In [ ]:
train.shape

### Plot que muestra los datos de entrenamiento y los de validación

In [ ]:
#plt.figure(figsize=(16,8))
plt.plot(train, label='Conjunto de entrenamiento (Training set): ' + str(len_train) +' puntos (80%)')
plt.plot(test, label='Conjunto de Validación (Validation set): '  + str(len_test) + ' puntos (20%)') 
plt.title("Serie de tiempo simulada, con la función seno y una tendencia", size = 20)
plt.legend()
plt.show()

Preparar los datos para la predicción de series temporales (LSTM en particular) puede ser complicado.

Intuitivamente, necesitamos predecir el valor en el paso de tiempo actual utilizando el historial ($n$ pasos de tiempo hacia atrás a partir de él). 


Aquí se propone una función genérica que hace el trabajo

In [ ]:
def create_dataset(X, y, time_steps=1):
    # crea dos listas vacias para depositar los datos
    Xs, ys = [], []
    # el primer lote de datos empieza en la primera observación
    # y toma time_steps  datos.
    # Comienza a avanzar hacia adelante.
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

La belleza de esta función es que trabaja con datos de series temporales univariadas (función única) y multivariadas (funciones múltiples). Usemos un historial de 50 pasos de tiempo para hacer nuestras secuencias. Esto significa que vamos a conservar la historia de 50 pasos atrás para predecir el valor actual.

In [ ]:
time_steps = 50

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train, time_steps)
X_test, y_test = create_dataset(test, test, time_steps)

print(X_train.shape, y_train.shape)

print([X_train[0:2,], y_train[0:2]])

In [ ]:
X_train.shape

## <span style="color:blue">Modelo LSTM</span>


Entrenar un modelo LSTM en Keras es fácil. Utilizaremos la capa LSTM en un modelo secuencial para hacer nuestras predicciones:

### Crea el modelo

In [ ]:
# shapes
inputs_shape = (X_train.shape[1], X_train.shape[2])
lstm_output = 60

# layers
inputs = Input(inputs_shape)
x = LSTM(units=lstm_output, name='LSTM_layer')(inputs)
outputs = Dense(1)(x)

# model
serie_0_1_model = Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')


### Summary del modelo

In [ ]:
#model = keras.Sequential()
#model.add(keras.layers.LSTM(units=lstm_output,  input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(keras.layers.Dense(units=1))
serie_0_1_model.summary()

plot_model(serie_0_1_model, to_file='../Imagenes/series_LSTM_model.png', 
           show_shapes=True)

Significa que el modelo tiene la primera de tipo LSTM capa con un input de tamaño $(50,1)$ y una salida de 60 neuronas.

La segunda es una capa densa con entrada 60 neuronas y salida una neurona


El cálculo del número de neuronas es como sigue: 

$$
\text{Número de parámetros capa LSTM } = 4(p^2+ pn +p)
$$

en donde $p$ es el tamaño de salida y $n$ el tamaño de entrada.

In [ ]:
lstm_output =60
input_size =1 # input size in the LSTM machine
#
# cada entrada de tamaño 50 es mostrada a la máquina  LSTM machine uno por uno.
# luego se tiene
#
num_params = 4*(lstm_output*lstm_output + lstm_output*input_size+ lstm_output)
num_params

### Compila

In [ ]:
serie_0_1_model.compile(loss='mean_squared_error',
  optimizer=Adam(0.001)
)

## Entrenamiento

Lo más importante para recordar al entrenar modelos de series temporales es no mezclar los datos (el orden de los datos es importante). El resto es bastante estándar:

In [ ]:
history = serie_0_1_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

## Evaluación del Modelo

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

In [ ]:
y_pred =serie_0_1_model.predict(X_test)

In [ ]:
#y_pred

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();



## Transforma a la escala original

In [ ]:
scaled_data = scaler.fit_transform(df1.values)
dataset = pd.DataFrame(scaled_data)
dataset.columns = ['sine']
dataset.index = df1.index

In [ ]:
y_pred = scaler.inverse_transform(y_pred)


In [ ]:
y_test = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
y_test.shape

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

## <span style="color:blue">Modelo GRU</span>

In [ ]:
# shapes
inputs_shape = (X_train.shape[1], X_train.shape[2])
lstm_output = 60

# layers
inputs = Input(inputs_shape)
x = GRU(units=lstm_output, name='GRU_layer')(inputs)
outputs = Dense(1)(x)

# model
serie_0_1_model_gru = Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')


# summary
serie_0_1_model_gru.summary()

plot_model(serie_0_1_model_gru, to_file='../Imagenes/series_LSTM_model.png', 
           show_shapes=True)

In [ ]:
# compila
serie_0_1_model_gru.compile(loss='mean_squared_error',
  optimizer=Adam(0.001)
)

In [ ]:
# entrena
history = serie_0_1_model_gru.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

In [ ]:
y_pred_gru =serie_0_1_model_gru.predict(X_test)
# escala original
y_pred_gru = scaler.inverse_transform(y_pred_gru)

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction lstm")
plt.plot(y_pred_gru, 'g', label="prediction gru")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();


In [ ]:
p= 60
n=1
3*(p*p +p*n+p)